# Setting

In [1]:
import sys, os
import pandas as pd

sys.path.append("../")
sys.path.append("../../")
sys.path.append("../../../")
sys.path.append("../../../../")

from Clust.setting import influx_setting_KETI as ins
from Clust.clust.ingestion.influx import influx_client_v2 as iC
from Clust.clust.ingestion.mongo import mongo_client
from Clust.clust.ML.common import ML_api
from Clust.clust.ML.tool import meta

influxdb_client = iC.InfluxClient(ins.CLUSTDataServer2)
mongodb_client = mongo_client.MongoClient(ins.CLUSTMetaInfo2)

/Users/kim-youngkee/Documents/CLUSTER/.clust/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cpu is available.


# 1. set param from Front End

In [2]:
# feature list와 data_y_flag는 model_info에서 받아와야 합니다.
# regression
params1 = {
    "ingestion_param_X" :{
        "bucket_name": 'integration',
        "ms_name" : 'regression_energy_cleanLevel4_testX',
        "feature_list": "None"
    },
    "ingestion_param_y":{
        "bucket_name": 'integration',
        "ms_name" : 'regression_energy_cleanLevel4_testy',
        "feature_list": "None"
    },
    'data_y_flag' : "None",
    'model_name':'regression_energy_cleanLevel4_trainX_regression_CNN_1D_rg_'
}

# forecasting
params2 = {
    "ingestion_param_X" :{
        "bucket_name": 'integration',
        "ms_name" : 'forecasting_Hs2SwineFarmWithWeatherTime_cleanLevel4_testX',
        "feature_list": "None"
    },
    "ingestion_param_y":{
        "bucket_name": 'integration',
        "ms_name" : 'forecasting_Hs2SwineFarmWithWeatherTime_cleanLevel4_testX',
        "feature_list": "None"
    },
    'data_y_flag' : "None",
    'model_name':'forecasting_Hs2SwineFarmWithWeatherTime_cleanLevel4_trainX_regression_LSTM_FCNs_rg_'
}

# classification
params3 = {
    "ingestion_param_X" :{
        "bucket_name": 'integration',
        "ms_name" : 'classification_actionPattern_cleanLevel0_testX',
        "feature_list": "None"
    },
    "ingestion_param_y":{
        "bucket_name": 'integration',
        "ms_name" : 'classification_actionPattern_cleanLevel0_testy',
        "feature_list": "None"
    },
    'data_y_flag' : "None",
    'model_name':'classification_actionPattern_cleanLevel0_trainX_classification_GRU_cf_'
}

# 2. Backend

In [3]:
# 1. Backend Parameter Setting
params = params3

# chage tpye string to bool -> ex) 'true' -> True
params = ML_api.chagne_type_str_to_bool(params)

In [4]:
# 2. get model meta
model_name = params['model_name']
meta_file_name = "./meta.json"
# for testing mongodb model
# model_meta = meta.read_model_meta_from_mongodb(mongodb_client, 'model', 'meta', model_name)
# for testing local model
model_meta = meta.read_model_meta_from_local(meta_file_name, model_name)

# data parameters from model meta
params['ingestion_param_X']['feature_list'] = model_meta['ingestion_param_X']['feature_list']
params['ingestion_param_y']['feature_list'] = model_meta['ingestion_param_y']['feature_list']
params['data_y_flag'] = model_meta['data_y_flag']

# scaler/transform info for test data preparation
params['scaler_param'] = model_meta['scaler_param']
params['transform_param'] = model_meta['transform_param']

# model info for ML test
params['model_info'] = model_meta['model_info']

In [5]:
# 3. Data Preparation
# make test data, scaler
test_X_array, test_y_array, scaler_X, scaler_y = ML_api.test_data_preparation(params, influxdb_client)

{'bucket_name': 'integration', 'ms_name': 'classification_actionPattern_cleanLevel0_testX', 'feature_list': ['col_0', 'col_1', 'col_2', 'col_3', 'col_4', 'col_5', 'col_6', 'col_7', 'col_8']}


window_size: 128 nan_limit_num: 64
(177280, 9) (1385, 128, 9)
(1385, 1) (1385, 1)


In [6]:
# set scaler by data y flag
if params['data_y_flag']:
    scaler = scaler_y
else:
    scaler = scaler_X

In [7]:
# model test
result = ML_api.ML_test(params, test_X_array, test_y_array, scaler)


Start testing data

** Performance of test dataset ==> PROB = [[9.5881861e-01 3.7298299e-02 1.3582031e-03 7.3410250e-04 1.1519705e-03
  6.3885981e-04]
 [7.8188270e-02 9.1924471e-01 7.1088778e-04 6.1114767e-04 7.3118613e-04
  5.1399641e-04]
 [9.5760530e-01 3.8397454e-02 1.4091459e-03 7.2831695e-04 1.1949097e-03
  6.6496589e-04]
 ...
 [9.5836687e-01 3.7675600e-02 1.3996233e-03 7.4431504e-04 1.1599541e-03
  6.5373356e-04]
 [9.5684290e-01 3.9098322e-02 1.4407958e-03 7.4634532e-04 1.2110701e-03
  6.6071784e-04]
 [9.6168762e-01 3.4093320e-02 1.6109215e-03 7.3177973e-04 1.2705321e-03
  6.0581567e-04]], ACC = 11.681686046511627
** Dimension of result for test dataset = (1376,)
scale
=========================data
      value  prediction
0       0.0         0.0
1       5.0         5.0
2       0.0         0.0
3       0.0         0.0
4       0.0         0.0
...     ...         ...
1371    0.0         0.0
1372    0.0         0.0
1373    0.0         0.0
1374    0.0         0.0
1375    0.0         0

In [8]:
result['result_metrics']

{'0': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 1139},
 '1': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 237},
 'accuracy': 1.0,
 'macro avg': {'precision': 1.0,
  'recall': 1.0,
  'f1-score': 1.0,
  'support': 1376},
 'weighted avg': {'precision': 1.0,
  'recall': 1.0,
  'f1-score': 1.0,
  'support': 1376}}

In [9]:
model_meta['model_info']

{'model_purpose': 'classification',
 'model_method': 'LSTM_FCNs_cf',
 'model_name': 'classification_actionPattern_cleanLevel0_trainX_classification_LSTM_FCNs_cf_',
 'model_tags': 'tagstest',
 'train_parameter': {'lr': 0.0001,
  'weight_decay': 1e-06,
  'n_epochs': 5,
  'batch_size': 16,
  'device': 'cpu'},
 'model_parameter': {'input_size': 9,
  'seq_len': 128,
  'num_classes': 6,
  'num_layers': 2,
  'lstm_dropout': 0.4,
  'fc_dropout': 0.1},
 'seq_len': 128,
 'input_size': 9,
 'model_file_path': {'modelFile': {'fileName': 'model.pth',
   'filePath': './Models/LSTM_FCNs_cf/classification_actionPattern_cleanLevel0_trainX_classification_LSTM_FCNs_cf_/classification_actionPattern_cleanLevel0_trainX/model.pkl'}}}